# Notebook Preprocesamiento
En el siguiente cuaderno se quiere mostrar que realizaciones y consideraciones se tuvieron en cuenta para aplicar las operaciones de limpieza y preprocesado de datos al archivo de `test.csv`.

## Cargar el Dataset

In [1]:
#Cargar el token de Kaggle
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"juangarcia140","key":"e2828f7c691979b2eb9ba44075d037b4"}'}

In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "."

In [ ]:
!kaggle competitions download udea-ai4eng-20241

 94% 20.0M/21.2M [00:01<00:00, 25.0MB/s]
100% 21.2M/21.2M [00:01<00:00, 14.9MB/s]


In [ ]:
#Descomprimiendo los archivos
!unzip udea-ai4eng-20241

Archive:  udea-ai4eng-20241.zip
  inflating: submission_example.csv  
  inflating: test.csv                
  inflating: train.csv               


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder

In [ ]:
# Cargar dataset

test = pd.read_csv('test.csv')

In [ ]:
test.head()

,ID,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_TIENELAVADORA,FAMI_TIENEAUTOMOVIL,ESTU_PRIVADO_LIBERTAD,ESTU_PAGOMATRICULAPROPIO,FAMI_TIENECOMPUTADOR,FAMI_TIENEINTERNET.1,FAMI_EDUCACIONMADRE
0,550236,20183,TRABAJO SOCIAL,BOLIVAR,Menos de 500 mil,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica completa,Si,No,N,Si,Si,Si,Primaria completa
1,98545,20203,ADMINISTRACION COMERCIAL Y DE MERCADEO,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 2,Si,Secundaria (Bachillerato) completa,Si,No,N,No,Si,Si,Técnica o tecnológica completa
2,499179,20212,INGENIERIA MECATRONICA,BOGOTÁ,Entre 1 millón y menos de 2.5 millones,0,Estrato 3,Si,Secundaria (Bachillerato) incompleta,Si,No,N,No,Si,Si,Secundaria (Bachillerato) completa
3,782980,20195,CONTADURIA PUBLICA,SUCRE,Entre 1 millón y menos de 2.5 millones,Entre 21 y 30 horas,Estrato 1,No,Primaria incompleta,Si,No,N,No,No,No,Primaria incompleta
4,785185,20212,ADMINISTRACION DE EMPRESAS,ATLANTICO,Entre 2.5 millones y menos de 4 millones,Entre 11 y 20 horas,Estrato 2,Si,Secundaria (Bachillerato) completa,Si,No,N,No,Si,Si,Secundaria (Bachillerato) completa


#Consideraciones
Del proceso de exploración de los datos, se infiere que el dataset en su composición tiene:

* Columnas repetidas:
__FAMI_TIENEINTERNET__ y __FAMI_TIENEINTERNET.1__

Adeás necesito categorizar varias columnas que representan un orden en su estructura
* ESTU_VALORMATRICULAUNIVERSIDAD
* ESTU_HORASSEMANATRABAJA
* FAMI_ESTRATOVIVIENDA
* FAMI_EDUCACIONPADRE
* FAMI_EDUCACIONMADRE

 Por lo que asignar numericamente un numero consecutivo, representa una categoria

In [ ]:
#Se hace la asignación, para el cambio de nombre de la columna repetida
test = test.rename(columns={'FAMI_TIENEINTERNET.1': 'FAMI_TIENEINTERNET2'})

In [ ]:
test.FAMI_TIENEINTERNET.value_counts()
#Columna original

FAMI_TIENEINTERNET
Si    254075
No     31172
Name: count, dtype: int64

In [ ]:
test.FAMI_TIENEINTERNET2.value_counts()
#COLUMNA DUPLICADA

FAMI_TIENEINTERNET2
Si    254075
No     31172
Name: count, dtype: int64

In [ ]:
validar = test['FAMI_TIENEINTERNET'].equals(test['FAMI_TIENEINTERNET2'])
print(validar)

True


In [ ]:
test = test.drop('FAMI_TIENEINTERNET2', axis=1)

In [ ]:
test.head()

,ID,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_TIENELAVADORA,FAMI_TIENEAUTOMOVIL,ESTU_PRIVADO_LIBERTAD,ESTU_PAGOMATRICULAPROPIO,FAMI_TIENECOMPUTADOR,FAMI_EDUCACIONMADRE
0,550236,20183,TRABAJO SOCIAL,BOLIVAR,Menos de 500 mil,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica completa,Si,No,N,Si,Si,Primaria completa
1,98545,20203,ADMINISTRACION COMERCIAL Y DE MERCADEO,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 2,Si,Secundaria (Bachillerato) completa,Si,No,N,No,Si,Técnica o tecnológica completa
2,499179,20212,INGENIERIA MECATRONICA,BOGOTÁ,Entre 1 millón y menos de 2.5 millones,0,Estrato 3,Si,Secundaria (Bachillerato) incompleta,Si,No,N,No,Si,Secundaria (Bachillerato) completa
3,782980,20195,CONTADURIA PUBLICA,SUCRE,Entre 1 millón y menos de 2.5 millones,Entre 21 y 30 horas,Estrato 1,No,Primaria incompleta,Si,No,N,No,No,Primaria incompleta
4,785185,20212,ADMINISTRACION DE EMPRESAS,ATLANTICO,Entre 2.5 millones y menos de 4 millones,Entre 11 y 20 horas,Estrato 2,Si,Secundaria (Bachillerato) completa,Si,No,N,No,Si,Secundaria (Bachillerato) completa


In [ ]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 296786 entries, 0 to 296785
Data columns (total 15 columns):
 #   Column                          Non-Null Count   Dtype 
---  ------                          --------------   ----- 
 0   ID                              296786 non-null  int64 
 1   PERIODO                         296786 non-null  int64 
 2   ESTU_PRGM_ACADEMICO             296786 non-null  object
 3   ESTU_PRGM_DEPARTAMENTO          296786 non-null  object
 4   ESTU_VALORMATRICULAUNIVERSIDAD  294063 non-null  object
 5   ESTU_HORASSEMANATRABAJA         283407 non-null  object
 6   FAMI_ESTRATOVIVIENDA            282991 non-null  object
 7   FAMI_TIENEINTERNET              285247 non-null  object
 8   FAMI_EDUCACIONPADRE             286793 non-null  object
 9   FAMI_TIENELAVADORA              279527 non-null  object
 10  FAMI_TIENEAUTOMOVIL             277868 non-null  object
 11  ESTU_PRIVADO_LIBERTAD           296786 non-null  object
 12  ESTU_PAGOMATRICULAPROPIO      

Evaluación general de todos los datos nulos

In [ ]:
k = test.isna().sum()
k[k!=0]


ESTU_VALORMATRICULAUNIVERSIDAD     2723
ESTU_HORASSEMANATRABAJA           13379
FAMI_ESTRATOVIVIENDA              13795
FAMI_TIENEINTERNET                11539
FAMI_EDUCACIONPADRE                9993
FAMI_TIENELAVADORA                17259
FAMI_TIENEAUTOMOVIL               18918
ESTU_PAGOMATRICULAPROPIO           2807
FAMI_TIENECOMPUTADOR              16439
FAMI_EDUCACIONMADRE               10223
dtype: int64

## Manejo de la columna PERIODO
Esta columna presenta 9 distintas posibilidades de ocurrencia, se quiere corregir y agrupar de mejor manera por año.

In [ ]:
test['PERIODO'].unique()

array([20183, 20203, 20212, 20195, 20196, 20194, 20213, 20202, 20184])

In [ ]:
test.PERIODO.value_counts()

PERIODO
20195    77579
20203    73918
20212    73415
20183    70406
20194      604
20213      492
20202      190
20184      119
20196       63
Name: count, dtype: int64

In [ ]:
test['PERIODO'].isna().sum()
#La columna periodo no posee datos nulos

0

In [ ]:
test['PERIODO'] = test['PERIODO'].astype('category')

In [ ]:
test['PERIODO'] = test['PERIODO'].astype(str).str.replace(r'2018.*', '2018', regex=True)
test['PERIODO'] = test['PERIODO'].astype(str).str.replace(r'2019.*', '2019', regex=True)
test['PERIODO'] = test['PERIODO'].astype(str).str.replace(r'2020.*', '2020', regex=True)
test['PERIODO'] = test['PERIODO'].astype(str).str.replace(r'2021.*', '2021', regex=True)

Ahora vamos a codificar esta columna


In [ ]:
#crear la instancia de OneHotEncoder
encode = OneHotEncoder()

#Trabaja con dataframes, no series
codificador = encode.fit_transform(test[['PERIODO']])
#generar codificacion

arreglo = codificador.toarray()
periodo = pd.DataFrame(arreglo,
                       columns = encode.categories_)

## Codificación de las columnas, manejo de OneHotEncoder


In [ ]:
test = pd.concat([test, periodo], axis = 'columns')



In [ ]:
test = test.drop('PERIODO', axis=1)

## Manejo de la columna ESTU_PRGM_ACADEMICO

In [ ]:
#La columna ESTU_PRGM_ACADEMICO es la que representa un mayor numero de variedad en el rango de sus posibilidades
test["ESTU_PRGM_ACADEMICO"].value_counts()

ESTU_PRGM_ACADEMICO
DERECHO                                                                22984
ADMINISTRACION DE EMPRESAS                                             22298
CONTADURIA PUBLICA                                                     16861
PSICOLOGIA                                                             13818
INGENIERIA INDUSTRIAL                                                  12516
                                                                       ...  
DISEÑO Y GESTIÓN DEL PRODUCTO                                              1
LICENCIATURA EN EDUCACION BASICA CON ENFASIS EN IDIOMAS EXTRANJEROS        1
ADMINISTRACION TECNOLOGICA                                                 1
LICENCIATURA EN EDUCACION ENFASIS EN HUMANIDADES                           1
AGROINDUSTRIA                                                              1
Name: count, Length: 919, dtype: int64

In [ ]:
test['ESTU_PRGM_ACADEMICO'].value_counts()

ESTU_PRGM_ACADEMICO
DERECHO                                                                22984
ADMINISTRACION DE EMPRESAS                                             22298
CONTADURIA PUBLICA                                                     16861
PSICOLOGIA                                                             13818
INGENIERIA INDUSTRIAL                                                  12516
                                                                       ...  
DISEÑO Y GESTIÓN DEL PRODUCTO                                              1
LICENCIATURA EN EDUCACION BASICA CON ENFASIS EN IDIOMAS EXTRANJEROS        1
ADMINISTRACION TECNOLOGICA                                                 1
LICENCIATURA EN EDUCACION ENFASIS EN HUMANIDADES                           1
AGROINDUSTRIA                                                              1
Name: count, Length: 919, dtype: int64

In [ ]:
!pip install unidecode

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 4.5 MB/s eta 0:00:00


Se hacen ciertas consideraciones para eliminar varios acentos, y caracteres es las posibles respuestas de la columna

In [ ]:
from unidecode import unidecode
import re

test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].str.upper() # Todo mayúsculas
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].apply(lambda x: unidecode(x).strip()) # Eliminar tildes
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].apply(lambda x: re.sub(r'\s+', ' ',x).strip()) # Eliminar dobles espacios
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].apply(lambda x: re.sub(r'^PROFESIONAL EN\s+', '', x).strip()) # "PROFESIONAL EN ..." redundante
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'INGENIER\?A', 'INGENIERIA', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'INGENIIERIA', 'INGENIERIA', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'EDUCACI\?N\b', 'EDUCACION', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'RECREACI\?N\b', 'RECREACION', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'F\?SICA\b', 'FISICA', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'ESC\?NICAS\b', 'ESCENICAS', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'MATEM\?TICAS\b', 'MATEMATICAS', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'FILOSOF\?A\b', 'FILOSOFIA', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'DISE\?O\b', 'DISEÑO', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'DISE\?\?O\b', 'DISEÑO', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'COMUNICACI\?N\b', 'COMUNICACION', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'COMUNICACI\?\?N\b', 'COMUNICACION', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'GR\?\?FICA\b', 'GRAFICA', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'EL\?CTRICA\b', 'ELECTRICA', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'QU\?MICA\b', 'QUIMICA', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'FARMAC\?UTICA\b', 'FARMACEUTICA', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'ADMINISTRACI\?N\b', 'ADMINISTRACION', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'P\?BLICA\b', 'PUBLICA', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'INGL\?S\b', 'INGLES', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'\?NFASIS\b', 'ENFASIS', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'GESTI\?N\b', 'GESTION', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'M\?SICA\b', 'MUSICA', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'ECONOM\?A\b', 'ECONOMIA', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'GASTRONOM\?A\b', 'GASTRONOMIA', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'B\?SICA\b', 'BASICA', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'GEOLOG\?A\b', 'GEOLOGIA', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'PEDAGOG\?A\b', 'PEDAGOGIA', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'ART\?STICA\b', 'ARTISTICA', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'LOG\?STICA\b', 'LOGISTICA', regex=True)

In [ ]:
test['ESTU_PRGM_ACADEMICO'].isnull().any()
#Se verifica que la columna no tenga valores nulos

False

In [ ]:
# Reemplazar cualquier coincidencia de "{NOMBRE + AGG} ..." solo por "{NOMBRE} mas general"
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'.*INGENIERIA.*', 'INGENIERIA', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'ADMINISTRACION.*', 'ADMINISTRACION', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'DISEÑO.*', 'DISEÑO', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'DISENO.*', 'DISEÑO', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'LICENCIATURA.*', 'LICENCIATURA', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'CIENCIA.*', 'CIENCIAS', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'COMUNICACION.*', 'COMUNICACION', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'GESTION.*', 'GESTION', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'FILOSOFIA.*', 'FILOSOFIA', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'QUIMICA.*', 'QUIMICA', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'NEGOCIOS.*', 'NEGOCIOS', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'.*MERCAD.*', 'MERCADEO', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'MATEMATICAS.*', 'MATEMATICAS', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'.*ARTE.*', 'ARTES', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'ESTUDIOS.*', 'ESTUDIOS', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'FINANZAS.*', 'FINANZAS', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'CONSTRUCCION.*', 'CONSTRUCCION', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'DIRECCION.*', 'DIRECCION', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'RELACIONES.*', 'RELACIONES', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'GEO.*', 'GEOGRAFIA', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'PROFESIONAL.*', 'PROFESIONAL', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'LENG.*', 'LENGUAS', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'ADMIN.*', 'ADMINISTRACION', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'.*DEPORT.*', 'DEPORTE', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'.*BIOLOGIA.*', 'BIOLOGIA', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'CINE.*', 'CINE', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'CONTADURIA.*', 'CONTADURIA', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'COMERCIO.*', 'COMERCIO', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'.*LITERA.*', 'LITERATURA', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'.*FILOL.*', 'FILOLOGIA', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'PUBLICI.*', 'PUBLICIDAD', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'MARK.*', 'MERCADEO', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'HISTORIA.*', 'HISTORIA', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'.*MUSICA.*', 'MUSICA', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'.*VETE.*', 'VETERINARIA', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'.*ZOO.*', 'VETERINARIA', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'.*GASTR.*', 'GASTRONOMIA', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'.*QUIRURGICA.*', 'MEDICINA', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'ECONO.*', 'ECONOMIA', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'GEREN.*', 'GERENCIA', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'HOTELERIA.*', 'HOTELERIA', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'.*PSICOLO.*', 'PSICOLOGIA', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'.*AGRO.*', 'AGRONOMIA', regex=True)


Lo que se hizo fue reducir el rango de las columna 'ESTU_PRGM_ACADEMICO', para que se generan muchas columnas al aplicar Codificación, mayormente llenas de ceros

In [ ]:
len(test['ESTU_PRGM_ACADEMICO'].unique())

109

In [ ]:
carreras_agrupadas = ['INGENIERIA','ADMINISTRACION','DISEÑO','LICENCIATURA','CIENCIAS','COMUNICACION','GESTION','FILOSOFIA','QUIMICA','NEGOCIOS','MERCADEO',
                       'MATEMATICAS','ARTES','ESTUDIOS','FINANZAS','CONSTRUCCION','DIRECCION','RELACIONES','GEOGRAFIA','PROFESIONAL','LENGUAS', 'DEPORTE'
                       'BIOLOGIA','CINE','CONTADURIA','COMERCIO','LITERATURA','FILOLOGIA','PUBLICIDAD','HISTORIA','MUSICA','VETERINARIA','GATRONOMIA'
                       'MEDICINA', 'ECONOMIA','GERENCIA','HOTELERIA','PSICOLOGIA','AGRONOMIA']
len(carreras_agrupadas)

37

In [ ]:
carreras_sobrantes = [elem for elem in test['ESTU_PRGM_ACADEMICO'] if elem not in carreras_agrupadas]

condicion = test['ESTU_PRGM_ACADEMICO'].isin(carreras_sobrantes)
test.loc[condicion, 'ESTU_PRGM_ACADEMICO'] = 'Otras'




In [ ]:
academico=test['ESTU_PRGM_ACADEMICO'].unique()
len(academico)
#academico

36

In [ ]:
##CODIFICAR PRGM ACADEMICO
#crear la instancia de OneHotEncoder
encode = OneHotEncoder()

#Trabaja con dataframes, no series
codificadorPr = encode.fit_transform(test[['ESTU_PRGM_ACADEMICO']])
#generar codificacion

arregloPr = codificadorPr.toarray()
programa = pd.DataFrame(arregloPr,
                       columns = encode.categories_)


In [ ]:
test = pd.concat([test, programa], axis = 'columns')


In [ ]:
test.head()

,ID,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_TIENELAVADORA,FAMI_TIENEAUTOMOVIL,...,"(MERCADEO,)","(MUSICA,)","(NEGOCIOS,)","(Otras,)","(PROFESIONAL,)","(PSICOLOGIA,)","(PUBLICIDAD,)","(QUIMICA,)","(RELACIONES,)","(VETERINARIA,)"
0,550236,Otras,BOLIVAR,Menos de 500 mil,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica completa,Si,No,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,98545,ADMINISTRACION,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 2,Si,Secundaria (Bachillerato) completa,Si,No,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,499179,INGENIERIA,BOGOTÁ,Entre 1 millón y menos de 2.5 millones,0,Estrato 3,Si,Secundaria (Bachillerato) incompleta,Si,No,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,782980,CONTADURIA,SUCRE,Entre 1 millón y menos de 2.5 millones,Entre 21 y 30 horas,Estrato 1,No,Primaria incompleta,Si,No,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,785185,ADMINISTRACION,ATLANTICO,Entre 2.5 millones y menos de 4 millones,Entre 11 y 20 horas,Estrato 2,Si,Secundaria (Bachillerato) completa,Si,No,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
test = test.drop('ESTU_PRGM_ACADEMICO', axis=1)

In [ ]:
ccols = [i for i in test.columns if not i in test._get_numeric_data()]
print (ccols)

['ESTU_PRGM_DEPARTAMENTO', 'ESTU_VALORMATRICULAUNIVERSIDAD', 'ESTU_HORASSEMANATRABAJA', 'FAMI_ESTRATOVIVIENDA', 'FAMI_TIENEINTERNET', 'FAMI_EDUCACIONPADRE', 'FAMI_TIENELAVADORA', 'FAMI_TIENEAUTOMOVIL', 'ESTU_PRIVADO_LIBERTAD', 'ESTU_PAGOMATRICULAPROPIO', 'FAMI_TIENECOMPUTADOR', 'FAMI_EDUCACIONMADRE']


In [ ]:
for c in ccols:
    print ("%10s"%c, np.unique(test[c].dropna()))

ESTU_PRGM_DEPARTAMENTO ['AMAZONAS' 'ANTIOQUIA' 'ARAUCA' 'ATLANTICO' 'BOGOTÁ' 'BOLIVAR' 'BOYACA'
 'CALDAS' 'CAQUETA' 'CASANARE' 'CAUCA' 'CESAR' 'CHOCO' 'CORDOBA'
 'CUNDINAMARCA' 'GUAVIARE' 'HUILA' 'LA GUAJIRA' 'MAGDALENA' 'META'
 'NARIÑO' 'NORTE SANTANDER' 'PUTUMAYO' 'QUINDIO' 'RISARALDA' 'SAN ANDRES'
 'SANTANDER' 'SUCRE' 'TOLIMA' 'VALLE' 'VAUPES']
ESTU_VALORMATRICULAUNIVERSIDAD ['Entre 1 millón y menos de 2.5 millones'
 'Entre 2.5 millones y menos de 4 millones'
 'Entre 4 millones y menos de 5.5 millones'
 'Entre 5.5 millones y menos de 7 millones'
 'Entre 500 mil y menos de 1 millón' 'Menos de 500 mil'
 'Más de 7 millones' 'No pagó matrícula']
ESTU_HORASSEMANATRABAJA ['0' 'Entre 11 y 20 horas' 'Entre 21 y 30 horas' 'Menos de 10 horas'
 'Más de 30 horas']
FAMI_ESTRATOVIVIENDA ['Estrato 1' 'Estrato 2' 'Estrato 3' 'Estrato 4' 'Estrato 5' 'Estrato 6'
 'Sin Estrato']
FAMI_TIENEINTERNET ['No' 'Si']
FAMI_EDUCACIONPADRE ['Educación profesional completa' 'Educación profesional incompleta'
 'Ni

## Manejo de la columna ESTU_PRGM_DEPARTAMENTO
Con el manejo de esta columna lo que se quiere es agrupar por regiones dichos departamentos, es decir, pasar de tener 32 tipos de departamentos, a solo cinco regiones que los agrupen

In [ ]:
test['ESTU_PRGM_DEPARTAMENTO'].unique()

array(['BOLIVAR', 'ANTIOQUIA', 'BOGOTÁ', 'SUCRE', 'ATLANTICO', 'VALLE',
       'SANTANDER', 'RISARALDA', 'QUINDIO', 'CASANARE', 'NORTE SANTANDER',
       'CAUCA', 'HUILA', 'CALDAS', 'CHOCO', 'CORDOBA', 'LA GUAJIRA',
       'META', 'NARIÑO', 'BOYACA', 'TOLIMA', 'CUNDINAMARCA', 'MAGDALENA',
       'CESAR', 'PUTUMAYO', 'CAQUETA', 'ARAUCA', 'AMAZONAS', 'VAUPES',
       'SAN ANDRES', 'GUAVIARE'], dtype=object)

In [ ]:
test['ESTU_PRGM_DEPARTAMENTO'] = test['ESTU_PRGM_DEPARTAMENTO'].astype(str).str.replace(r'ANTIOQUIA', 'ANDINA')
test['ESTU_PRGM_DEPARTAMENTO'] = test['ESTU_PRGM_DEPARTAMENTO'].astype(str).str.replace(r'BOYACA', 'ANDINA')
test['ESTU_PRGM_DEPARTAMENTO'] = test['ESTU_PRGM_DEPARTAMENTO'].astype(str).str.replace(r'BOGOTÁ', 'ANDINA')
test['ESTU_PRGM_DEPARTAMENTO'] = test['ESTU_PRGM_DEPARTAMENTO'].astype(str).str.replace(r'CALDAS', 'ANDINA')
test['ESTU_PRGM_DEPARTAMENTO'] = test['ESTU_PRGM_DEPARTAMENTO'].astype(str).str.replace(r'CUNDINAMARCA', 'ANDINA')
test['ESTU_PRGM_DEPARTAMENTO'] = test['ESTU_PRGM_DEPARTAMENTO'].astype(str).str.replace(r'HUILA', 'ANDINA')
test['ESTU_PRGM_DEPARTAMENTO'] = test['ESTU_PRGM_DEPARTAMENTO'].astype(str).str.replace(r'NORTE SANTANDER', 'ANDINA')
test['ESTU_PRGM_DEPARTAMENTO'] = test['ESTU_PRGM_DEPARTAMENTO'].astype(str).str.replace(r'QUINDIO', 'ANDINA')
test['ESTU_PRGM_DEPARTAMENTO'] = test['ESTU_PRGM_DEPARTAMENTO'].astype(str).str.replace(r'RISARALDA', 'ANDINA')
test['ESTU_PRGM_DEPARTAMENTO'] = test['ESTU_PRGM_DEPARTAMENTO'].astype(str).str.replace(r'SANTANDER', 'ANDINA')
test['ESTU_PRGM_DEPARTAMENTO'] = test['ESTU_PRGM_DEPARTAMENTO'].astype(str).str.replace(r'TOLIMA', 'ANDINA')

In [ ]:
test['ESTU_PRGM_DEPARTAMENTO'] = test['ESTU_PRGM_DEPARTAMENTO'].astype(str).str.replace(r'AMAZONAS', 'AMAZONA')
test['ESTU_PRGM_DEPARTAMENTO'] = test['ESTU_PRGM_DEPARTAMENTO'].astype(str).str.replace(r'CAQUETA', 'AMAZONA')
test['ESTU_PRGM_DEPARTAMENTO'] = test['ESTU_PRGM_DEPARTAMENTO'].astype(str).str.replace(r'VAUPES', 'AMAZONA')
test['ESTU_PRGM_DEPARTAMENTO'] = test['ESTU_PRGM_DEPARTAMENTO'].astype(str).str.replace(r'GUAVIARE', 'AMAZONA')
test['ESTU_PRGM_DEPARTAMENTO'] = test['ESTU_PRGM_DEPARTAMENTO'].astype(str).str.replace(r'PUTUMAYO', 'AMAZONA')

In [ ]:
test['ESTU_PRGM_DEPARTAMENTO'] = test['ESTU_PRGM_DEPARTAMENTO'].astype(str).str.replace(r'VALLE', 'PACIFICA')
test['ESTU_PRGM_DEPARTAMENTO'] = test['ESTU_PRGM_DEPARTAMENTO'].astype(str).str.replace(r'CHOCO', 'PACIFICA')
test['ESTU_PRGM_DEPARTAMENTO'] = test['ESTU_PRGM_DEPARTAMENTO'].astype(str).str.replace(r'CAUCA', 'PACIFICA')
test['ESTU_PRGM_DEPARTAMENTO'] = test['ESTU_PRGM_DEPARTAMENTO'].astype(str).str.replace(r'NARIÑO', 'PACIFICA')

In [ ]:
test['ESTU_PRGM_DEPARTAMENTO'] = test['ESTU_PRGM_DEPARTAMENTO'].astype(str).str.replace(r'ATLANTICO', 'CARIBE')
test['ESTU_PRGM_DEPARTAMENTO'] = test['ESTU_PRGM_DEPARTAMENTO'].astype(str).str.replace(r'BOLIVAR', 'CARIBE')
test['ESTU_PRGM_DEPARTAMENTO'] = test['ESTU_PRGM_DEPARTAMENTO'].astype(str).str.replace(r'CESAR', 'CARIBE')
test['ESTU_PRGM_DEPARTAMENTO'] = test['ESTU_PRGM_DEPARTAMENTO'].astype(str).str.replace(r'CORDOBA', 'CARIBE')
test['ESTU_PRGM_DEPARTAMENTO'] = test['ESTU_PRGM_DEPARTAMENTO'].astype(str).str.replace(r'LA GUAJIRA', 'CARIBE')
test['ESTU_PRGM_DEPARTAMENTO'] = test['ESTU_PRGM_DEPARTAMENTO'].astype(str).str.replace(r'MAGDALENA', 'CARIBE')
test['ESTU_PRGM_DEPARTAMENTO'] = test['ESTU_PRGM_DEPARTAMENTO'].astype(str).str.replace(r'SUCRE', 'CARIBE')
test['ESTU_PRGM_DEPARTAMENTO'] = test['ESTU_PRGM_DEPARTAMENTO'].astype(str).str.replace(r'SAN ANDRES', 'CARIBE')

In [ ]:
test['ESTU_PRGM_DEPARTAMENTO'] = test['ESTU_PRGM_DEPARTAMENTO'].astype(str).str.replace(r'ARAUCA', 'ORINOQUIA')
test['ESTU_PRGM_DEPARTAMENTO'] = test['ESTU_PRGM_DEPARTAMENTO'].astype(str).str.replace(r'CASANARE', 'ORINOQUIA')
test['ESTU_PRGM_DEPARTAMENTO'] = test['ESTU_PRGM_DEPARTAMENTO'].astype(str).str.replace(r'META', 'ORINOQUIA')
test['ESTU_PRGM_DEPARTAMENTO'] = test['ESTU_PRGM_DEPARTAMENTO'].astype(str).str.replace(r'VICHADA', 'ORINOQUIA')

In [ ]:
test['ESTU_PRGM_DEPARTAMENTO'].unique()

array(['CARIBE', 'ANDINA', 'PACIFICA', 'ORINOQUIA', 'AMAZONA'],
      dtype=object)

In [ ]:
test['ESTU_PRGM_DEPARTAMENTO'].isna().sum()

0

In [ ]:
##CODIFICAR PRGM DEPARTAMENTO
#crear la instancia de OneHotEncoder
encode = OneHotEncoder()

#Trabaja con dataframes, no series
codificadorDepto = encode.fit_transform(test[['ESTU_PRGM_DEPARTAMENTO']])
#generar codificacion

arregloDepto = codificadorDepto.toarray()
departamento = pd.DataFrame(arregloDepto,
                       columns = encode.categories_)

In [ ]:
#Suma de dataframes
test = pd.concat([test, departamento], axis = 'columns')

In [ ]:
test.head()

,ID,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_TIENELAVADORA,FAMI_TIENEAUTOMOVIL,ESTU_PRIVADO_LIBERTAD,...,"(PSICOLOGIA,)","(PUBLICIDAD,)","(QUIMICA,)","(RELACIONES,)","(VETERINARIA,)","(AMAZONA,)","(ANDINA,)","(CARIBE,)","(ORINOQUIA,)","(PACIFICA,)"
0,550236,CARIBE,Menos de 500 mil,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica completa,Si,No,N,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,98545,ANDINA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 2,Si,Secundaria (Bachillerato) completa,Si,No,N,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,499179,ANDINA,Entre 1 millón y menos de 2.5 millones,0,Estrato 3,Si,Secundaria (Bachillerato) incompleta,Si,No,N,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,782980,CARIBE,Entre 1 millón y menos de 2.5 millones,Entre 21 y 30 horas,Estrato 1,No,Primaria incompleta,Si,No,N,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,785185,CARIBE,Entre 2.5 millones y menos de 4 millones,Entre 11 y 20 horas,Estrato 2,Si,Secundaria (Bachillerato) completa,Si,No,N,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [ ]:
test = test.drop('ESTU_PRGM_DEPARTAMENTO', axis=1)

## Manejo de la columna ESTU_VALORMATRICULAUNIVERSIDAD
La columna presenta una posibilidad de 8 valores a la hroa de elegir cualquier registro y 6287 datos son nulos

In [ ]:
test['ESTU_VALORMATRICULAUNIVERSIDAD'].unique()


array(['Menos de 500 mil', 'Entre 2.5 millones y menos de 4 millones',
       'Entre 1 millón y menos de 2.5 millones',
       'Entre 500 mil y menos de 1 millón', 'Más de 7 millones',
       'Entre 4 millones y menos de 5.5 millones',
       'Entre 5.5 millones y menos de 7 millones', 'No pagó matrícula',
       nan], dtype=object)

In [ ]:
test['ESTU_VALORMATRICULAUNIVERSIDAD'].value_counts()

ESTU_VALORMATRICULAUNIVERSIDAD
Entre 1 millón y menos de 2.5 millones      86923
Entre 2.5 millones y menos de 4 millones    55193
Menos de 500 mil                            34589
Entre 500 mil y menos de 1 millón           33564
Entre 4 millones y menos de 5.5 millones    29855
Más de 7 millones                           29061
Entre 5.5 millones y menos de 7 millones    16558
No pagó matrícula                            8320
Name: count, dtype: int64

In [ ]:
test['ESTU_VALORMATRICULAUNIVERSIDAD'].isna().sum()

2723

Tenemos una columna que presenta 8 posibles categorias, y que la categoria con mayor frecuencia, es un valor de matricula entre 1 y 2.5 M.

Se decide rellenar los datos faltantes, asumiendo que aquellos estudiantes que no tengan un valor asignado, es como si no realizaran un pago


In [ ]:
test['ESTU_VALORMATRICULAUNIVERSIDAD'] = test['ESTU_VALORMATRICULAUNIVERSIDAD'].fillna('No pagó matrícula')

In [ ]:
#Realizamos el Encode para este columna
valores_matricula = ['No pagó matrícula','Menos de 500 mil','Entre 500 mil y menos de 1 millón',
                     'Entre 1 millón y menos de 2.5 millones',
                     'Entre 2.5 millones y menos de 4 millones',
                     'Entre 4 millones y menos de 5.5 millones',
                     'Entre 5.5 millones y menos de 7 millones','Más de 7 millones']

enc = OrdinalEncoder(categories = [valores_matricula])

test['ESTU_VALORMATRICULAUNIVERSIDAD'] = enc.fit_transform(test[['ESTU_VALORMATRICULAUNIVERSIDAD']])

In [ ]:
test['ESTU_VALORMATRICULAUNIVERSIDAD'].value_counts()

ESTU_VALORMATRICULAUNIVERSIDAD
3.0    86923
4.0    55193
1.0    34589
2.0    33564
5.0    29855
7.0    29061
6.0    16558
0.0    11043
Name: count, dtype: int64

## Manejo de la columna ESTU_HORASSEMANATRABAJA
La columna presenta 5 categorias, y presenta 30857 datos nulos, por lo que no debo realizar alguna reparacion, paar este caso asumimos que si el campo esta vacio, es por que el estudiante no tiene unas horas de trabajo


In [ ]:
test['ESTU_HORASSEMANATRABAJA'].unique()

array(['Menos de 10 horas', 'Entre 21 y 30 horas', '0',
       'Entre 11 y 20 horas', 'Más de 30 horas', nan], dtype=object)

In [ ]:
test['ESTU_HORASSEMANATRABAJA'].value_counts()

ESTU_HORASSEMANATRABAJA
Más de 30 horas        106189
0                       50422
Entre 11 y 20 horas     49386
Entre 21 y 30 horas     39931
Menos de 10 horas       37479
Name: count, dtype: int64

In [ ]:
test['ESTU_HORASSEMANATRABAJA'].isna().sum()

13379

In [ ]:
test['ESTU_HORASSEMANATRABAJA'] = test['ESTU_HORASSEMANATRABAJA'].fillna('0')

In [ ]:
horas_trabajo = ['0','Menos de 10 horas','Entre 11 y 20 horas',
                'Entre 21 y 30 horas',
                'Más de 30 horas']

enc = OrdinalEncoder(categories = [horas_trabajo])

test['ESTU_HORASSEMANATRABAJA'] = enc.fit_transform(test[['ESTU_HORASSEMANATRABAJA']])

In [ ]:
test['ESTU_HORASSEMANATRABAJA'].value_counts()

ESTU_HORASSEMANATRABAJA
4.0    106189
0.0     63801
2.0     49386
3.0     39931
1.0     37479
Name: count, dtype: int64

## Manejo de la columna FAMI_ESTRATOVIVIENDA
Dicha columna presenta 7 categorias, y ademas de eso un total de 32137 datos nulos, la estrategia que se plantea para repaar dichos datos nulos se baso en ver cual era la categoria con mayor frecuencia

In [ ]:
test['FAMI_ESTRATOVIVIENDA'].unique()

array(['Estrato 3', 'Estrato 2', 'Estrato 1', 'Estrato 4', nan,
       'Estrato 6', 'Estrato 5', 'Sin Estrato'], dtype=object)

In [ ]:
test['FAMI_ESTRATOVIVIENDA'].value_counts()

FAMI_ESTRATOVIVIENDA
Estrato 2      99288
Estrato 3      90719
Estrato 1      48092
Estrato 4      27823
Estrato 5      10194
Estrato 6       5451
Sin Estrato     1424
Name: count, dtype: int64

In [ ]:
test['FAMI_ESTRATOVIVIENDA'].isna().sum()

13795

In [ ]:
test['FAMI_ESTRATOVIVIENDA'] = test['FAMI_ESTRATOVIVIENDA'].fillna('Estrato 2')

In [ ]:
estratos = ['Sin Estrato','Estrato 1','Estrato 2',
            'Estrato 3', 'Estrato 4', 'Estrato 5',
            'Estrato 6']

enc = OrdinalEncoder(categories = [estratos])

test['FAMI_ESTRATOVIVIENDA'] = enc.fit_transform(test[['FAMI_ESTRATOVIVIENDA']])

In [ ]:
test['FAMI_ESTRATOVIVIENDA'].value_counts()

FAMI_ESTRATOVIVIENDA
2.0    113083
3.0     90719
1.0     48092
4.0     27823
5.0     10194
6.0      5451
0.0      1424
Name: count, dtype: int64

## Manejo de la columna FAMI_TIENEINTERNET
Para este caso, tengo solo dos posibles clases, por lo que debo codificar de manera binaria, ademas tengo 26629 datos nuelos, y se decide clasificar dichos datos que no aparecen, como un 'NO'

In [ ]:
test['FAMI_TIENEINTERNET'].unique()

array(['Si', 'No', nan], dtype=object)

In [ ]:
test['FAMI_TIENEINTERNET'].value_counts()

FAMI_TIENEINTERNET
Si    254075
No     31172
Name: count, dtype: int64

In [ ]:
test['FAMI_TIENEINTERNET'].isna().sum()

11539

In [ ]:
test['FAMI_TIENEINTERNET'] = test['FAMI_TIENEINTERNET'].fillna("No")

In [ ]:
test['FAMI_TIENEINTERNET'] = test['FAMI_TIENEINTERNET'].replace(r'Si', 1.)
test['FAMI_TIENEINTERNET'] = test['FAMI_TIENEINTERNET'].replace(r'No', 0.)

In [ ]:
test['FAMI_TIENEINTERNET'].value_counts()

FAMI_TIENEINTERNET
1.0    254075
0.0     42711
Name: count, dtype: int64

## Manejo de la columna FAMI_TIENELAVADORA
Esta columna tambien presenta una posibilidad de valores de 2 rangos, por lo que dicha encodificación va a ser binaria, ademas contienen 39773 registros nulos

In [ ]:
test['FAMI_TIENELAVADORA'].unique()

array(['Si', nan, 'No'], dtype=object)

In [ ]:
test['FAMI_TIENELAVADORA'].value_counts()

FAMI_TIENELAVADORA
Si    241565
No     37962
Name: count, dtype: int64

In [ ]:
test['FAMI_TIENELAVADORA'].isna().sum()

17259

In [ ]:
test['FAMI_TIENELAVADORA'] = test['FAMI_TIENELAVADORA'].fillna("No")

In [ ]:
test['FAMI_TIENELAVADORA'] = test['FAMI_TIENELAVADORA'].replace(r'Si', 1.)
test['FAMI_TIENELAVADORA'] = test['FAMI_TIENELAVADORA'].replace(r'No', 0.)

In [ ]:
test['FAMI_TIENELAVADORA'].value_counts()

FAMI_TIENELAVADORA
1.0    241565
0.0     55221
Name: count, dtype: int64

## Manejo de la columna FAMI_TIENEAUTOMOVIL
La columna presenta una distinción entre valores de tipo binario, por lo que todos los datos pueden tomar solo dos valores, y presenta 43623 registros nulos

In [ ]:
test['FAMI_TIENEAUTOMOVIL'].unique()

array(['No', 'Si', nan], dtype=object)

In [ ]:
test['FAMI_TIENEAUTOMOVIL'].value_counts()

FAMI_TIENEAUTOMOVIL
No    176625
Si    101243
Name: count, dtype: int64

In [ ]:
test['FAMI_TIENEAUTOMOVIL'].isna().sum()

18918

In [ ]:
test['FAMI_TIENEAUTOMOVIL'] = test['FAMI_TIENEAUTOMOVIL'].fillna("No")

In [ ]:
test['FAMI_TIENEAUTOMOVIL'] = test['FAMI_TIENEAUTOMOVIL'].replace(r'Si', 1.)
test['FAMI_TIENEAUTOMOVIL'] = test['FAMI_TIENEAUTOMOVIL'].replace(r'No', 0.)

In [ ]:
test['FAMI_TIENEAUTOMOVIL'].value_counts()

FAMI_TIENEAUTOMOVIL
0.0    195543
1.0    101243
Name: count, dtype: int64

## Manejo de la columna FAMI_TIENECOMPUTADOR
Comportmaiento binario.
Presenta 38103 registros nulos

In [ ]:
test['FAMI_TIENECOMPUTADOR'].unique()

array(['Si', 'No', nan], dtype=object)

In [ ]:
test['FAMI_TIENECOMPUTADOR'].value_counts()

FAMI_TIENECOMPUTADOR
Si    255996
No     24351
Name: count, dtype: int64

In [ ]:
test['FAMI_TIENECOMPUTADOR'].isna().sum()

16439

In [ ]:
test['FAMI_TIENECOMPUTADOR'] = test['FAMI_TIENECOMPUTADOR'].fillna("No")

In [ ]:
test['FAMI_TIENECOMPUTADOR'] = test['FAMI_TIENECOMPUTADOR'].replace(r'Si', 1.)
test['FAMI_TIENECOMPUTADOR'] = test['FAMI_TIENECOMPUTADOR'].replace(r'No', 0.)

In [ ]:
test['FAMI_TIENECOMPUTADOR'].value_counts()

FAMI_TIENECOMPUTADOR
1.0    255996
0.0     40790
Name: count, dtype: int64

## Manejo de la columna ESTU_PAGOMATRICULAPROPIO
Esta columna presenta un rango binario en su conjunto de posibles valores, y un total de 6498 datos nulos.
Por el momento la decisión para hcaer la reparación, se elige de asignar 'NO' a los datos nulos

In [ ]:
test['ESTU_PAGOMATRICULAPROPIO'].unique()

array(['Si', 'No', nan], dtype=object)

In [ ]:
test['ESTU_PAGOMATRICULAPROPIO'].value_counts()

ESTU_PAGOMATRICULAPROPIO
No    163869
Si    130110
Name: count, dtype: int64

In [ ]:
test['ESTU_PAGOMATRICULAPROPIO'].isna().sum()

2807

In [ ]:
test['ESTU_PAGOMATRICULAPROPIO'] = test['ESTU_PAGOMATRICULAPROPIO'].fillna("No")

In [ ]:
test['ESTU_PAGOMATRICULAPROPIO'] = test['ESTU_PAGOMATRICULAPROPIO'].replace(r'Si', 1.)
test['ESTU_PAGOMATRICULAPROPIO'] = test['ESTU_PAGOMATRICULAPROPIO'].replace(r'No', 0.)

In [ ]:
test['ESTU_PAGOMATRICULAPROPIO'].value_counts()

ESTU_PAGOMATRICULAPROPIO
0.0    166676
1.0    130110
Name: count, dtype: int64

## Manejo de la columna ESTU_PRIVADO_LIBERTAD
Esta columna tiene un comportamiento binario, y no presenta datos nulos, se tiene pendiente el proceso de evaluar la presencia de esta variable en el modelo, ya que el 99% de los datos son registros con negación de la condición de la columna

In [ ]:
test['ESTU_PRIVADO_LIBERTAD'].unique()

array(['N', 'S'], dtype=object)

In [ ]:
test['ESTU_PRIVADO_LIBERTAD'].value_counts()

ESTU_PRIVADO_LIBERTAD
N    296773
S        13
Name: count, dtype: int64

In [ ]:
test['ESTU_PRIVADO_LIBERTAD'].isna().sum()

0

In [ ]:
test['ESTU_PRIVADO_LIBERTAD'] = test['ESTU_PRIVADO_LIBERTAD'].replace(r'S', 1.)
test['ESTU_PRIVADO_LIBERTAD'] = test['ESTU_PRIVADO_LIBERTAD'].replace(r'N', 0.)

In [ ]:
test['ESTU_PRIVADO_LIBERTAD'].value_counts()

ESTU_PRIVADO_LIBERTAD
0.0    296773
1.0        13
Name: count, dtype: int64

## Manejo de la COLUMNA FAMI_EDUCACIONPADRE
Esta columna presento varias modificaciones, entre ellas la agrupación de las etiquetas 'Ninguno', 'No sabe' y 'No aplica' en una sola, junto con lso datos nulos, ademas de eso, se considera importante el hecho de jerarquizar dichas etiquetas

In [ ]:
test['FAMI_EDUCACIONPADRE'].unique()

array(['Técnica o tecnológica completa',
       'Secundaria (Bachillerato) completa',
       'Secundaria (Bachillerato) incompleta', 'Primaria incompleta',
       'Ninguno', 'Primaria completa', 'Postgrado', nan,
       'Educación profesional completa',
       'Técnica o tecnológica incompleta',
       'Educación profesional incompleta', 'No sabe', 'No Aplica'],
      dtype=object)

In [ ]:
test['FAMI_EDUCACIONPADRE'].value_counts()

FAMI_EDUCACIONPADRE
Secundaria (Bachillerato) completa      54838
Primaria incompleta                     53432
Educación profesional completa          35412
Secundaria (Bachillerato) incompleta    30641
Técnica o tecnológica completa          27045
Primaria completa                       24131
Postgrado                               19287
Educación profesional incompleta        11744
Técnica o tecnológica incompleta         9677
Ninguno                                  9565
No sabe                                  7066
No Aplica                                3955
Name: count, dtype: int64

* 0 No aplica
* 1 primaria incompleta
* 2 primaria completa
* 3 Secundaria (Bachillerato) incompleta  
* 4 Secundaria (Bachillerato) completa
* 5 Tecnica o tecnolgia incompleta
* 6 Tecnica o tecnolgia incompleta
* 7 Educacion profesional incompleta
* 8 Educacion profesional completa
* 9 postgrado


In [ ]:
test['FAMI_EDUCACIONPADRE'].isna().sum()

9993

In [ ]:
test['FAMI_EDUCACIONPADRE'] = test['FAMI_EDUCACIONPADRE'].replace(r'Ninguno', 'No Aplica')
test['FAMI_EDUCACIONPADRE'] = test['FAMI_EDUCACIONPADRE'].replace(r'No sabe', 'No Aplica')

In [ ]:
test['FAMI_EDUCACIONPADRE'] = test['FAMI_EDUCACIONPADRE'].fillna('No Aplica')

In [ ]:
test['FAMI_EDUCACIONPADRE'].value_counts()

FAMI_EDUCACIONPADRE
Secundaria (Bachillerato) completa      54838
Primaria incompleta                     53432
Educación profesional completa          35412
Secundaria (Bachillerato) incompleta    30641
No Aplica                               30579
Técnica o tecnológica completa          27045
Primaria completa                       24131
Postgrado                               19287
Educación profesional incompleta        11744
Técnica o tecnológica incompleta         9677
Name: count, dtype: int64

Ahora toca codificar la columna

In [ ]:

nivelesPadre = ['No Aplica','Primaria incompleta','Primaria completa',
            'Secundaria (Bachillerato) incompleta', 'Secundaria (Bachillerato) completa', 'Técnica o tecnológica incompleta',
            'Técnica o tecnológica completa', 'Educación profesional incompleta', 'Educación profesional completa',
            'Postgrado']

enc = OrdinalEncoder(categories = [nivelesPadre])

test['FAMI_EDUCACIONPADRE'] = enc.fit_transform(test[['FAMI_EDUCACIONPADRE']])

In [ ]:
test['FAMI_EDUCACIONPADRE'].value_counts()

FAMI_EDUCACIONPADRE
4.0    54838
1.0    53432
8.0    35412
3.0    30641
0.0    30579
6.0    27045
2.0    24131
9.0    19287
7.0    11744
5.0     9677
Name: count, dtype: int64

In [ ]:
test.head()

,ID,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_TIENELAVADORA,FAMI_TIENEAUTOMOVIL,ESTU_PRIVADO_LIBERTAD,ESTU_PAGOMATRICULAPROPIO,...,"(PSICOLOGIA,)","(PUBLICIDAD,)","(QUIMICA,)","(RELACIONES,)","(VETERINARIA,)","(AMAZONA,)","(ANDINA,)","(CARIBE,)","(ORINOQUIA,)","(PACIFICA,)"
0,550236,1.0,1.0,3.0,1.0,6.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,98545,4.0,3.0,2.0,1.0,4.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,499179,3.0,0.0,3.0,1.0,3.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,782980,3.0,3.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,785185,4.0,2.0,2.0,1.0,4.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


## Manejo de la columna FAMI_EDUCACIONMADRE
Esta columna presento varias modificaciones, entre ellas la agrupación de las etiquetas 'Ninguno', 'No sabe' y 'No aplica' en una sola, ademas de eso, se considera importante el hecho de jerarquizar dichas etiquetas

In [ ]:
test['FAMI_EDUCACIONMADRE'].unique()

array(['Primaria completa', 'Técnica o tecnológica completa',
       'Secundaria (Bachillerato) completa', 'Primaria incompleta',
       'Postgrado', 'Secundaria (Bachillerato) incompleta', nan,
       'Ninguno', 'Educación profesional completa',
       'Técnica o tecnológica incompleta',
       'Educación profesional incompleta', 'No sabe', 'No Aplica'],
      dtype=object)

In [ ]:
test['FAMI_EDUCACIONMADRE'].value_counts()

FAMI_EDUCACIONMADRE
Secundaria (Bachillerato) completa      60649
Primaria incompleta                     42149
Técnica o tecnológica completa          38571
Educación profesional completa          36477
Secundaria (Bachillerato) incompleta    34566
Primaria completa                       24399
Postgrado                               20049
Técnica o tecnológica incompleta        11650
Educación profesional incompleta         9647
Ninguno                                  6335
No sabe                                  1286
No Aplica                                 785
Name: count, dtype: int64

In [ ]:
test['FAMI_EDUCACIONMADRE'].isna().sum()

10223

In [ ]:
test['FAMI_EDUCACIONMADRE'] = test['FAMI_EDUCACIONMADRE'].replace(r'Ninguno', 'No Aplica')
test['FAMI_EDUCACIONMADRE'] = test['FAMI_EDUCACIONMADRE'].replace(r'No sabe', 'No Aplica')

In [ ]:
test['FAMI_EDUCACIONMADRE'] = test['FAMI_EDUCACIONMADRE'].fillna('No Aplica')

In [ ]:
nivelesMadre = ['No Aplica','Primaria incompleta','Primaria completa',
            'Secundaria (Bachillerato) incompleta', 'Secundaria (Bachillerato) completa', 'Técnica o tecnológica incompleta',
            'Técnica o tecnológica completa', 'Educación profesional incompleta', 'Educación profesional completa',
            'Postgrado']

enc = OrdinalEncoder(categories = [nivelesMadre])

test['FAMI_EDUCACIONMADRE'] = enc.fit_transform(test[['FAMI_EDUCACIONMADRE']])

In [ ]:
test['FAMI_EDUCACIONMADRE'].value_counts()

FAMI_EDUCACIONMADRE
4.0    60649
1.0    42149
6.0    38571
8.0    36477
3.0    34566
2.0    24399
9.0    20049
0.0    18629
5.0    11650
7.0     9647
Name: count, dtype: int64

In [ ]:
test.head()

,ID,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_TIENELAVADORA,FAMI_TIENEAUTOMOVIL,ESTU_PRIVADO_LIBERTAD,ESTU_PAGOMATRICULAPROPIO,...,"(PSICOLOGIA,)","(PUBLICIDAD,)","(QUIMICA,)","(RELACIONES,)","(VETERINARIA,)","(AMAZONA,)","(ANDINA,)","(CARIBE,)","(ORINOQUIA,)","(PACIFICA,)"
0,550236,1.0,1.0,3.0,1.0,6.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,98545,4.0,3.0,2.0,1.0,4.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,499179,3.0,0.0,3.0,1.0,3.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,782980,3.0,3.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,785185,4.0,2.0,2.0,1.0,4.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [ ]:
test.shape

(296786, 57)

In [ ]:
print(len(test.columns))
test.columns

57


Index([                            'ID', 'ESTU_VALORMATRICULAUNIVERSIDAD',
              'ESTU_HORASSEMANATRABAJA',           'FAMI_ESTRATOVIVIENDA',
                   'FAMI_TIENEINTERNET',            'FAMI_EDUCACIONPADRE',
                   'FAMI_TIENELAVADORA',            'FAMI_TIENEAUTOMOVIL',
                'ESTU_PRIVADO_LIBERTAD',       'ESTU_PAGOMATRICULAPROPIO',
                 'FAMI_TIENECOMPUTADOR',            'FAMI_EDUCACIONMADRE',
                              ('2018',),                        ('2019',),
                              ('2020',),                        ('2021',),
                    ('ADMINISTRACION',),                   ('AGRONOMIA',),
                             ('ARTES',),                    ('CIENCIAS',),
                              ('CINE',),                    ('COMERCIO',),
                      ('COMUNICACION',),                ('CONSTRUCCION',),
                        ('CONTADURIA',),                   ('DIRECCION',),
                         

In [ ]:
archivo = test.to_csv("testP1.csv", index = False)

In [ ]:
!head testP1.csv

ID,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_TIENELAVADORA,FAMI_TIENEAUTOMOVIL,ESTU_PRIVADO_LIBERTAD,ESTU_PAGOMATRICULAPROPIO,FAMI_TIENECOMPUTADOR,FAMI_EDUCACIONMADRE,"('2018',)","('2019',)","('2020',)","('2021',)","('ADMINISTRACION',)","('AGRONOMIA',)","('ARTES',)","('CIENCIAS',)","('CINE',)","('COMERCIO',)","('COMUNICACION',)","('CONSTRUCCION',)","('CONTADURIA',)","('DIRECCION',)","('DISEÑO',)","('ECONOMIA',)","('ESTUDIOS',)","('FILOLOGIA',)","('FILOSOFIA',)","('FINANZAS',)","('GEOGRAFIA',)","('GERENCIA',)","('GESTION',)","('HISTORIA',)","('HOTELERIA',)","('INGENIERIA',)","('LENGUAS',)","('LICENCIATURA',)","('LITERATURA',)","('MATEMATICAS',)","('MERCADEO',)","('MUSICA',)","('NEGOCIOS',)","('Otras',)","('PROFESIONAL',)","('PSICOLOGIA',)","('PUBLICIDAD',)","('QUIMICA',)","('RELACIONES',)","('VETERINARIA',)","('AMAZONA',)","('ANDINA',)","('CARIBE',)","('ORINOQUIA',)","('PACIFICA',)"
550236,1.0,1.0,3.0,1.0,6.0,